In [ ]:
# !pip install protobuf feast[aws] redis

In [ ]:
# run feast ui in console:
# feast ui -p 8081

In [ ]:
cd feature_store

In [ ]:
# feast will read all files in the directory so need to delete checkpoints
!rm -rf .ipynb_checkpoints

In [ ]:
!feast apply

In [ ]:
from feast import FeatureStore

store = FeatureStore(repo_path=".")

## Offline queries

In [ ]:
import feast
import pandas as pd
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage

entity_df = pd.DataFrame.from_dict({"HouseId": [i for i in range(1, 1000)]})
entity_df['event_timestamp'] = pd.to_datetime('now', utc=True)

retrieval_job = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service('house_service')
)

In [ ]:
retrieval_job.metadata.features

In [ ]:
dataset = store.create_saved_dataset(
    from_=retrieval_job,
    name='merged_dataset_v1',
    storage=SavedDatasetFileStorage('s3://dvoitekh-kubeflow/feast/data/merged_dataset_v1.parquet')
)

training_df = retrieval_job.to_df()
print(len(training_df))
training_df.head()

In [ ]:
!feast materialize-incremental 2022-02-01T00:00:00

## Online Queries

In [ ]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

In [ ]:
!feast materialize-incremental 2022-06-11T00:00:00

In [ ]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

## Push Source

In [ ]:
import datetime

store.push('main_push_source', pd.DataFrame([{
    'HouseId': 1,
    'MedInc': 1.0,
    'HouseAge': 1.0,
    'AveRooms': 1.0,
    'AveBedrms': 1.0,
    'Population': 1.0,
    'AveOccup': 1.0,
    'MedHouseVal': 1.0,
    'Latitude': 1.0,
    'Longitude': 1.0,
    'EventTimestamp': datetime.datetime.now(),
    'Created': datetime.datetime.now(),
}]))

In [ ]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

In [ ]:
entity_df = pd.DataFrame.from_dict({"HouseId": [1]})
entity_df['event_timestamp'] = pd.to_datetime('now', utc=True)

store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service('house_service')
).to_df()